### 工具包导入（可不看）

In [1]:
#!/usr/bin/env python
# coding:utf-8
# 导入工具包
import pandas as pd
import selenium 
import io
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options  
import time
from selenium.webdriver.common.by import By
# 导入下拉框选择模块
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
# 添加元素加载等待模块
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# 导入运动链
from selenium.webdriver.common.action_chains import ActionChains
# 获取店铺代码
import os
# 页面登陆
from selenium.webdriver import ChromeOptions
import requests
import re
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from io import BytesIO
import requests
from hashlib import md5
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import urllib.parse 
import requests
import io 
import json 
from PIL import Image
from selenium.webdriver.chrome.service import Service
# Use the function
# 浏览器初始化
def init_driver(chrome_exe_path, chromedriver_path):
    """
    Initialize and return a Selenium WebDriver instance.

    Parameters:
    - chrome_exe_path: The path to the Chrome executable.
    - chromedriver_path: The path to the chromedriver executable.

    Returns:
    - A Selenium WebDriver instance.
    """
    # Specify the path to the Chrome executable
    __browser_url = chrome_exe_path

    # Create a new Chrome Options object
    chrome_options = Options()

    # Add options
    chrome_options.add_argument("--disable-blink-features")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
    chrome_options.binary_location = __browser_url


    s = Service(chromedriver_path)
    driver = webdriver.Chrome(service=s,options=chrome_options)

    # Visit the specified URL
    driver.get('http://admin.digitalretail.chjgroup.cn/admin/user/login')

    # Maximize the window
    driver.maximize_window()

    return driver
# 完成登陆操作
def login_to_website(driver):
    driver.get('http://admin.digitalretail.chjgroup.cn/admin/user/login')
    driver.maximize_window()
    driver.find_element(By.XPATH, "/html/body/div/div[2]/form/div[1]/input").send_keys('595838518@qq.com')
    driver.find_element(By.XPATH, "/html/body/div/div[2]/form/div[2]/input").send_keys('595838518@qq.com')
    captcha_img_element = driver.find_element(By.XPATH, '/html/body/div/div[2]/form/div[3]/div[1]/img')
    captcha_img_data = captcha_img_element.screenshot_as_png
    captcha_img = Image.open(BytesIO(captcha_img_data))
    with open('captcha.jpg', 'wb') as file:
        file.write(captcha_img_data)
    chaojiying = Chaojiying_Client('DanielXie', 'X1i2e3y4u5', '63ccfdfa9ab27e3014b96521de087458')
    im = open('captcha.jpg', 'rb').read()	
    num = chaojiying.PostPic(im,4004)['pic_str']	
    driver.find_element(By.XPATH, "/html/body/div/div[2]/form/div[3]/div[2]/input").send_keys(num)
    driver.find_element(By.XPATH, "/html/body/div/div[2]/form/div[4]/button").click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/aside/section/ul/li[3]/a/span[1]")))
# 获取总共的页面数
def select_and_search_product(driver):
    driver.get('http://admin.digitalretail.chjgroup.cn/admin/mall/new/product?first_catalog_id=18&is_deleted=0&keyword=')
    # 获取末页元素，从中提取最大页码
    last_page_element = driver.find_element(By.XPATH, '//ul[@class="pagination pagination-sm no-margin pull-right"]/li/a[contains(text(), "末页")]')
    last_page_href = last_page_element.get_attribute('href')
    last_page = int(last_page_href.split('=')[-1])  # 从末页链接中提取最大页码
    # 将所有页的url拼接起来
    page_urls = [f'http://admin.digitalretail.chjgroup.cn/admin/mall/new/product?first_catalog_id=18&is_deleted=0&keyword=&page={page}' for page in range(9, last_page + 1)]
    return page_urls
# 超级鹰验证码模块
class Chaojiying_Client(object):

    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def PostPic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()

    def PostPic_base64(self, base64_str, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
            'file_base64':base64_str
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, headers=self.headers)
        return r.json()

    def ReportError(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()
# 读取文件
def process_data(data2):
    # 将第一个表的数据导入到第二个表中
    data2['仓库代码'] = 'STZBZC'
    data2.rename(columns={'商品代码': '69码/版库号'}, inplace=True)
    data2['库存'] = data2['门店实时库存（剔除未核销）'] - data2['实时上架库存（包含虚拟）'].astype('int')
    data2['类型（实际库存，虚拟库存，虚拟替换）'] = '虚拟库存'
    data2['快递公司'] = ''
    data2['快递单号'] = ''
    data2['收件人联系方式'] = ''
    data2['备注'] = ''
    data2['子订单号'] = ''
    data2.drop(['实时上架库存（包含虚拟）', '门店实时库存（剔除未核销）', '门店实物库存（包含未核销）', '期初实物库存数', '在途数',
                '本月入库实物数', '本月入库虚拟数', '本月出库数', '本月下单数', '本月之前下单未核销数',
                '本月下单未核销数', '本月下单已核销数', '本月盘盈亏数'], axis=1, inplace=True)
    data2 = data2[['仓库代码', '门店代码', '69码/版库号', '库存', '类型（实际库存，虚拟库存，虚拟替换）', '快递公司',
                   '快递单号', '收件人联系方式', '备注', '子订单号']]

    return data2
# 商品详情
def get_product_info(title, cookies_dict):
    base_url = 'http://admin.digitalretail.chjgroup.cn/admin/mall/gift_new/stock_export?shopCode=&title_sku={}&v=1'
    title_sku = urllib.parse.quote(title) # 进行URL编码
    url = base_url.format(title_sku)
    response = requests.get(url, cookies=cookies_dict)
    # 将response.content转化为csv格式并读取到df中
    if response.content.decode('utf-8'):
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    else:
        df = pd.DataFrame()
    error_data = df[df['门店实时库存（剔除未核销）'] - df['实时上架库存（包含虚拟）'] != 0]
    # 首列中得''
    error_data = error_data[error_data['门店实时库存（剔除未核销）'] != '']
    error_data['门店代码'] = error_data['门店代码'].astype('str')
    # 用replace把门店代码列中得''字符串去掉
    error_data['门店代码'] = error_data['门店代码'].str.replace("''", '')
    # 保存文件，这里用产品名称作为文件名，请确保它是合法的文件名
    try:
        error_data = process_data(error_data)
        error_data.to_excel(f"./data/{title}.xlsx", index=False)
    except:
        return None
    return error_data
# 文件上传
def upload_file(driver, title):
    # Open the target URL
    driver.get('http://admin.digitalretail.chjgroup.cn/admin/mall/new/product_store_list')
    # Click on the upload button
    upload_btn_xpath = '/html/body/div/div[1]/section[2]/div/div[1]/div/div/form/a[1]'
    driver.find_element(By.XPATH, upload_btn_xpath).click()
    # Get the absolute path of the current folder
    folder_path = os.path.abspath('.')

    # Construct the full file path with the given title
    file_path = os.path.join(folder_path, 'data', f'{title}.xlsx')

    # Enter the file path into the input field
    input_xpath = '/html/body/div[1]/div[2]/div/div/form/div[1]/div/input'
    driver.find_element(By.XPATH, input_xpath).send_keys(file_path)

    # Click the upload button using explicit wait
    upload_button_xpath = '/html/body/div[1]/div[2]/div/div/form/div[2]/div/button[2]'
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, upload_button_xpath))
    ).click()
        # 跳转页面至http://admin.digitalretail.chjgroup.cn/admin/mall/new/product_store_list?date_start=&v=1&date_end=&bill_no=&createBy=rpa&out_store_no=&store_no=&goods_code=&billType=2&status=0&to_sap=0&v=1
    driver.get('http://admin.digitalretail.chjgroup.cn/admin/mall/new/product_store_list?date_start=&v=1&date_end=&bill_no=&createBy=rpa&out_store_no=&store_no=&goods_code=&billType=2&status=0&to_sap=0&v=1')
    # 点击一键审核，xpath是/html/body/div/div[1]/section[2]/div/div[1]/div/div/form/a[3]
    audit_xpath = '/html/body/div/div[1]/section[2]/div/div[1]/div/div/form/a[3]'
    driver.find_element(By.XPATH, audit_xpath).click()
    # 点击确定，xpath是/html/body/div[2]/div/div/div[2]/button[2]
    confirm_xpath = '/html/body/div[2]/div/div/div[2]/button[2]'
    WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, confirm_xpath))
    ).click()
    # 用ec判断是否成功后点击ok，xpath是这个/html/body/div[2]/div/div/div[2]/button
    ok_xpath = '/html/body/div[2]/div/div/div[2]/button'
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, ok_xpath))
    ).click()
    # 页面刷新
    time.sleep(5)
    # 跳转审核
    driver.execute_script("location.reload()")
    driver.get('http://admin.digitalretail.chjgroup.cn/admin/mall/new/product_store_list?date_start=&v=1&date_end=&bill_no=&createBy=rpa&out_store_no=&store_no=&goods_code=&billType=2&status=1&to_sap=0&v=1')    
    # 点击一键验收，xpath是/html/body/div/div[1]/section[2]/div/div[1]/div/div/form/a[4]
    check_xpath = '/html/body/div/div[1]/section[2]/div/div[1]/div/div/form/a[4]'
    # 用ec来点击
    try:
        WebDriverWait(driver, ).until(
            EC.element_to_be_clickable((By.XPATH, check_xpath))
        ).click()
    except:
        driver.get('http://admin.digitalretail.chjgroup.cn/admin/mall/new/product_store_list?date_start=&v=1&date_end=&bill_no=&createBy=rpa&out_store_no=&store_no=&goods_code=&billType=2&status=1&to_sap=0&v=1')
        WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, check_xpath))
        ).click()
    # 点击确定，xpath是/html/body/div[2]/div/div/div[2]/button[2]
    confirm_xpath = '/html/body/div[2]/div/div/div[2]/button[2]'
    # 用ec来点击
    WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, confirm_xpath))
    ).click()
    # 用ec判断是否成功后点击ok，xpath是这个/html/body/div[2]/div/div/div[2]/button
    ok_xpath = '/html/body/div[2]/div/div/div[2]/button'
    WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, ok_xpath))
    ).click()


# 工具说明
本工具包主要应用于礼品科的礼品库存状态更新，主要功能分为以下几个部分：

### 1. 真实库存礼品批量识别并批量处理
    a. 通过运行此功能模块可快速整理识别现阶段所有的礼品中存在真实库存状态的礼品，并存储到《现有真实库存名单.xlsx》
    b. 将现有所有真实库存名单的所有礼品进行更新

### 2. 临时存库礼品信息更新
    a. 读取《临时处理礼品.xlsx》中人工导入的礼品名称
    b. 批量处理临时处理的真实库存礼品信息

## 1. 真实库存礼品批量识别并批量处理

步骤：在下方方框中按住ctrl+enter进行运行

In [ ]:
driver = init_driver(r'C:\Users\Administrator\AppData\Local\360Chrome\Chrome\Application\360chrome.exe', 'C:\Program Files\Google\Chrome\Application\chromedriver.exe')
login_to_website(driver)
page_urls = select_and_search_product(driver)
real_product = []
driver.close()
for page_url in page_urls:
    # 提取当前页数
    page_num = int(page_url.split('=')[-1])
    driver = init_driver(r'C:\Users\Administrator\AppData\Local\360Chrome\Chrome\Application\360chrome.exe', 'C:\Program Files\Google\Chrome\Application\chromedriver.exe')
    login_to_website(driver)
    try:
        driver.get(page_url)
    except:
        driver.refresh()
        driver.get(page_url)
    tr_elements = driver.find_elements(By.CLASS_NAME, 'border_bottom')
    edit_hrefs = []
    product_names = []
    # 遍历tr_elements，获取当页的产品名称和编辑链接
    for tr_element in tr_elements:
        # 找到编辑按钮并获取其href属性
        href = tr_element.find_element(By.XPATH, './td[9]/div[1]/p[1]/a[1]').get_attribute('href')
        # 将href添加到edit_hrefs列表中
        edit_hrefs.append(href)
        # 找到商品名称并获取其文本
        product_name = tr_element.find_element(By.XPATH, './td[2]/div/div[2]/p[1]/b').text
        # 将商品名称添加到product_names列表中
        product_names.append(product_name)
    # 跳转编辑链接并查看是否有真实库存
    # 刷新一下当前页面
    for edit_href in edit_hrefs:
        # 刷新一下当前页面
        driver.get(edit_href)
        # 帮我写一个xpath来查找'真实库存'
        input_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/section[2]/div/div[2]/div[2]/div[2]/div[1]/form/div[8]/div/div/div/div/div[1]/input"))
        )
        # 获取input元素的值
        input_value = input_element.get_attribute('value')
        # 如果文本是真实库存，就将商品名称添加到real_product列表中
        if input_value == '真实库存':
            real_product.append(product_names[edit_hrefs.index(edit_href)])
        else:
            continue
    # 导出每一个页现有的真实库存名单到df中
    tmp_df = pd.DataFrame(real_product)
    # 将df导出到excel中,标注好是第几页的数据
    tmp_df.to_excel('第' + str(page_num) + '页现有真实库存名单.xlsx')
    # 关闭浏览器
    driver.close()

# 用os库读取D:\code\python\礼品科自动化 这个文件夹中的所有文件
files = os.listdir()
# 用for循环遍历所有文件
# 创建一个空的df
df = pd.DataFrame()

for file in files:
    # 如果文件名中包含xlsx，就将其读取到df中
    if 'xlsx' in file:
        tmpdf = pd.read_excel(file)
        # 将所有的tmpdf合并到df中
        df = pd.concat([df, tmpdf], ignore_index=True)
driver = init_driver(r'C:\Users\Administrator\AppData\Local\360Chrome\Chrome\Application\360chrome.exe', 'C:\Program Files\Google\Chrome\Application\chromedriver.exe')
login_to_website(driver)
cookies = driver.get_cookies()
cookies_dict = {}
for cookie in cookies:
    cookies_dict[cookie['name']] = cookie['value']
df = df.iloc[:, 1].drop_duplicates()    
# 导出df到excel中
df.to_excel('现有真实库存名单.xlsx')
df = pd.DataFrame()
for idx, row in df.iterrows():
    title = row['真实库存']
    tmp_df = get_product_info(title, cookies_dict)
    upload_file(driver,title)
driver.close()

## 2. 临时存库礼品信息更新
步骤：
1. 将所有的想要修改的礼品填入文件夹中的《临时处理礼品.xlsx》中
2. 在下方方框中按住ctrl+enter进行运行

In [ ]:
driver = init_driver(r'C:\Users\xieyu\AppData\Local\Google\Chrome\Application\chrome.exe', r'C:\Users\xieyu\AppData\Local\Google\Chrome\Application\chromedriver.exe')
login_to_website(driver)
cookies = driver.get_cookies()
cookies_dict = {}
for cookie in cookies:
    cookies_dict[cookie['name']] = cookie['value']
# 循环获取所有产品信息 
data = pd.read_excel(r'D:\code\潮宏基\日常脚本\gift_rpa\临时处理礼品.xlsx')
# 创建一个空的df
df = pd.DataFrame()
for idx, row in data.iterrows():
    title = row['真实库存']
    try:
        tmp_df = get_product_info(title, cookies_dict)
        print("上传成功",title)
    except:
        print("上传失败",title)
        continue
    try:
        upload_file(driver,title)
    except:
        print("上传失败",title)
        continue
driver.close()